### Test for websockets_manager

In [1]:
import asyncio
import json
import logging
import websockets
# from custom_types import PriceLevel
from typing import List, Dict, Any, Optional
from websockets_manager import WebsocketManager
from orderbook import OrderBook

logger = logging.getLogger(__name__)

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("CLOB_API_KEY")
API_SECRET = os.getenv("CLOB_SECRET")
API_PASSPHRASE = os.getenv("CLOB_PASS_PHRASE")
AGENCY_ADDRESS = os.getenv("AGENCY_ADDRESS")

YES_ID = "46171502957028595838694604499378583099478369877679255114043087599679007467964"
NO_ID = "21457067342029925682172110629429793886827168158271370485467219837949936707498"
CONDITION_ID = "0x07ab34a38692464baf22c7e859550712bb2093fe005b24c58a0717f6e9595e55"

MARKET_WSS_ENDPOINT = "wss://ws-subscriptions-clob.polymarket.com/ws/market"
USER_WSS_ENDPOINT = "wss://ws-subscriptions-clob.polymarket.com/ws/user"

In [3]:
book = OrderBook(CONDITION_ID, YES_ID, NO_ID)

In [4]:
# the closure
def create_market_handler(order_book: OrderBook, yes_id: str = YES_ID):
    async def handler(data):
        for item in data:
            if item.get('asset_id') == yes_id:
                if item.get('event_type') == 'book':
                    order_book.update_book(item.get('bids'), item.get('asks'))
                elif item.get('event_type') == 'price_change':
                    order_book.handle_price_change(item.get('changes'))
    return handler

    # print(f"Market data received: {data}")

async def example_user_handler(data):
    print(f"User data received: {data}")

In [5]:
manager = WebsocketManager(
    market_url=MARKET_WSS_ENDPOINT,
    user_url=USER_WSS_ENDPOINT,
    api_key=API_KEY,
    api_secret=API_SECRET,
    api_passphrase=API_PASSPHRASE,
    condition_id=CONDITION_ID,
    yes_id=YES_ID,
    no_id=NO_ID
)

manager.add_market_handler(create_market_handler(book))
manager.add_user_handler(example_user_handler)

In [6]:
try:
    await manager.start()
except KeyboardInterrupt:
    # logger.info("Shutting down...")
    await manager.stop()

Sent market subscription: {'assets_ids': ['46171502957028595838694604499378583099478369877679255114043087599679007467964', '21457067342029925682172110629429793886827168158271370485467219837949936707498'], 'type': 'market'}
Sent user subscription: {'auth': {'apiKey': '689c3290-d89e-a3ba-4548-c21125c1efab', 'secret': 'HLmOg9rCRe046UMqgHmTPHfGyz9GVSoFRDw9w4E1Y0U=', 'passphrase': 'e28daa72920b234eeb1f9b6bf19cb62318f96dea2545bd2ec56f4985d788d853'}, 'markets': ['0x07ab34a38692464baf22c7e859550712bb2093fe005b24c58a0717f6e9595e55'], 'type': 'user'}


CancelledError: 